# Extracting and visualising a free energy simulation

This notebook provides a step-by-step guide to extract and visualise a free energy simulation trajectory from a ``simulation.nc`` file using [openfe-analysis](https://github.com/OpenFreeEnergy/openfe_analysis) and [MDAnalysis](https://github.com/MDAnalysis/mdanalysis). By the end, you should understand how to:

1. Extract the trajectory of a ``replica`` or ``single lambda state`` from a ``simulation.nc`` file
2. Centre the ligand in the simulation box using `MDAnalysis`
3. Write out the trajectorie(s) using `MDAnalysis`

## Downloading the example data

First, download some example trajectory data. This may take a few minutes due to the size of the simulation file. Please skip this section if you have already done this!

In [1]:
! wget https://zenodo.org/records/15375081/files/simulation.nc
! wget https://zenodo.org/records/15375081/files/hybrid_system.pdb

--2025-09-05 10:31:50--  https://zenodo.org/records/15375081/files/simulation.nc
Resolving zenodo.org (zenodo.org)... 2001:1458:d00:25::100:372, 2001:1458:d00:24::100:f6, 2001:1458:d00:61::100:2f3, ...
Connecting to zenodo.org (zenodo.org)|2001:1458:d00:25::100:372|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 516886878 (493M) [application/octet-stream]
Saving to: ‘simulation.nc.1’

simulation.nc.1     100%[===================>] 492.94M  12.2MB/s    in 62s     

2025-09-05 10:32:53 (7.96 MB/s) - ‘simulation.nc.1’ saved [516886878/516886878]

--2025-09-05 10:32:53--  https://zenodo.org/records/15375081/files/hybrid_system.pdb
Resolving zenodo.org (zenodo.org)... 2001:1458:d00:24::100:f6, 2001:1458:d00:61::100:2f3, 2001:1458:d00:25::100:372, ...
Connecting to zenodo.org (zenodo.org)|2001:1458:d00:24::100:f6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 388547 (379K) [application/octet-stream]
Saving to: ‘hybrid_system.pdb.1’

hybrid

## Extracting the trajectory with `MDAnalysis`

The `openfe-analysis` package provides an `MDAnalysis` reader to help extract the trajectory data from the `simulation.nc` file. As the file contains multipule replicas simulated at different lambda states, we must choose which of these to load as a single trajectory. We have two options available to construct the trajectory:
- `state_id`: will construct a trajectory which follows a single Hamiltonian lambda state at the specified value.
- `recplica_id`: will construct a trajectory which follows a single replica at the specified value.

In this example which uses a trajectory from a relative binding free energy calculation we will load the trajectory at `lambda=0` or the end state corresponding to Ligand A and visulaise the trajectory with `nglview`.

In [6]:
import MDAnalysis as mda
from openfe_analysis import FEReader
import nglview as nv

u_0 = mda.Universe("hybrid_system.pdb", "simulation.nc", format=FEReader, state_id=0)

w = nv.show_mdanalysis(u_0)
w

/Users/joshua/mambaforge/envs/openfe_fast_set/lib/python3.12/site-packages/openfe_analysis/utils/multistate.py:41: UserWarning: This is an older NetCDF file that does not yet contain information about the write frequency of positions and velocities. We will assume that positions and velocities were written out at every iteration. 
  warnings.warn(wmsg)


NGLWidget(max_frame=500)

<div class=\"alert alert-block alert-info\"> <b>Note:</b> The OpenFE relative binding free energy protocol does not save water positions by default, this can be changed via the <a href="https://docs.openfree.energy/en/latest/reference/api/openmm_protocol_settings.html#openfe.protocols.openmm_utils.omm_settings.MultiStateOutputSettings.output_indices">output_indices</a> protocol setting. </div>


To view the final state at `lambda=1` we can use negative indexing if we don't know the total number of lambda states.

In [7]:
u_1 = mda.Universe("hybrid_system.pdb", "simulation.nc", format=FEReader, state_id=-1)

w = nv.show_mdanalysis(u_1)
w.center_view()
w

/Users/joshua/mambaforge/envs/openfe_fast_set/lib/python3.12/site-packages/openfe_analysis/utils/multistate.py:41: UserWarning: This is an older NetCDF file that does not yet contain information about the write frequency of positions and velocities. We will assume that positions and velocities were written out at every iteration. 
  warnings.warn(wmsg)
DEPRECATED: Please use 'center' method


NGLWidget(max_frame=500)

## Centring the Ligand

You may have noticed in the view above that the ligand seems to have drifted away from the protein, this is a visualisation artifact caused by the use of periodic boundary conditions and the way in which OpenMM tries to ensure that all particle positions are written into a single periodic box. We can fix this however, using transformations in `MDAnalysis` to centre the ligand and wrap the protein around it like so:

In [8]:
import MDAnalysis.transformations as trans

ligand = u_0.select_atoms("resname UNK")
protein = u_0.select_atoms("protein")
transforms = [
    trans.center_in_box(ligand, wrap=True),
    trans.wrap(protein),
    trans.nojump.NoJump()
]
u_0.trajectory.add_transformations(*transforms)

w = nv.show_mdanalysis(u_0)
w

NGLWidget(max_frame=500)

## Saving the trajectory to file

We can now use `MDAnalysis` to save our trajectory to a common file format:

In [9]:
with mda.Writer('out.xtc', n_atoms=len(u_0.atoms)) as w:
    for ts in u_0.trajectory:
        w.write(u_0.atoms)